In [1]:
import sqlite3
conn = sqlite3.connect("db.sqlite3")
cursor = conn.cursor()

In [2]:
def write_to_sqlite(info):
    sql = ''' insert into article_articlepost
            (title, body, comments, create_time, favors, likes,passage_url, pre_id, tag, abs, author_id)
        values
              (:title, :body, :comments, :create_time, :favors, :likes, :passage_url, :pre_id, :tag, :abs, :author_id)'''
    cursor.execute(sql,{'title':info['passage_title'], 'body':info['body'], 'comments':info['comments'], 'create_time':info['create_time'], 'favors':0, 'likes':info['likes'], 'passage_url':info['passage_url'], 'pre_id':info['user_name'], 'tag': info['tag'], 'abs':info['passage_abs'], 'author_id':1})
    conn.commit()

In [7]:
import os
# os.mkdir('./result')

In [9]:
pre_dataset = os.listdir('./result')
print(len(pre_dataset))

5506


In [13]:
import json
print(pre_dataset[0])
current_data_dir = './result/'+pre_dataset[0]
current_html = current_data_dir+'/body.html'
result_json = current_data_dir + '/result.json'
if os.path.exists(result_json) :
    with open(result_json, 'r') as f:
        result = json.load(f)
        # print(result)
        result = result.get('results')
    

1069


In [18]:
url_list = []
for i in range(0,len(pre_dataset)):
    # print("Now: ", i)
    current_data_dir = './result/'+pre_dataset[i]
    current_html = current_data_dir+'/body.html'
    result_json = current_data_dir + '/result.json'
    if os.path.exists(result_json) :
        with open(result_json, 'r') as f:
            result = json.load(f)
            # print(result)
            result = result.get('results')
            url = result['passage_url']
            if url in url_list :
                # print("重复")
                continue
            else:
                url_list += [result['passage_url']]
                if result['body'] == 'None':
                    print(result_json)
                    print(None)
                    pass
                else:
                    # write_to_sqlite(result)
                    pass
                    
print(len(url_list))

./result/4973/result.json
None
./result/1381/result.json
None
./result/5137/result.json
None
./result/5488/result.json
None
./result/1659/result.json
None
./result/4746/result.json
None
./result/5175/result.json
None
5224


In [34]:
import os
import requests
import json
import fake_useragent
import re
from bs4 import BeautifulSoup as bs
headers = {'User-Agent': fake_useragent.UserAgent().random}
cookie = '_uab_collina=166002750457757950766442; web_login_version=MTY2MjM4ODk4NA==--1d65d71599c072230cf859ea0395607843632db4; _m7e_session_core=1b78529f65f906accd5519fd1692ac7d; read_mode=day; default_font=font2; locale=zh-CN; sensorsdata2015jssdkcross={"distinct_id":"17c63de9bf3a60-05e7d57d8cf78f-1d3b6650-1296000-17c63de9bf4d8f","first_id":"","props":{"$latest_traffic_source_type":"直接流量","$latest_search_keyword":"未取到值_直接打开","$latest_referrer":"","$latest_utm_source":"desktop","$latest_utm_medium":"not-signed-in-collection-follow-button"},"$device_id":"17c63de9bf3a60-05e7d57d8cf78f-1d3b6650-1296000-17c63de9bf4d8f"}; acw_tc=0bcd4ce216626530430792163e0158022bd8033591f4046d711089fbe56fa6; acw_sc__v2=631a127338fb43417d15ef17ae708cc76647ef5e; acw_sc__v3=631a148b1c795d68c50dacee14c10b177e3b7cce; signin_redirect=https://www.jianshu.com/u/96287c50380a?u_atoken=1b1c28de-7005-477e-a484-3c68508fb289&u_asession=01Pq3QKNzbhGoqv_FQ7j016aRYPhPBPVQfrmiFwBg_00GD9CDPoNPVVbYZkEp0s-voX0KNBwm7Lovlpxjd_P_q4JsKWYrT3W_NKPr8w6oU7K-65cm5DoPF8LVcY-_34t58zdjoMV1y19BFQvaXcOyBfmBkFo3NEHBv0PZUm6pbxQU&u_asig=051MWtxGbwrW4aSPSBXhEshe9GXA_6ZRjl2Ac6o0uJHZBzGUouoSup1uTamWfxypqP4tF2__yjNzVB1Clxw7U6n4lMFk6Si9iNCYRuExcfkk01RAPYKyWOKJFQlholl_wXnpSxRKq8YIRNluI_SgchTYaQFCoSfSFAuwIq1tH20V79JS7q8ZD7Xtz2Ly-b0kmuyAKRFSVJkkdwVUnyHAIJzYS3LP3ATrfOuVA2yfpKYmKfm1tR-rihidxC4LaeQTUBqBR97QLsOYcZJeUxi-_JXu3h9VXwMyh6PgyDIVSG1W-DQQxdxEPgdtXqY0uK2gVtBqzEBEQjIpjZO2BpBfmomhK--xgf0p8Il673vd9F3zriAn15EJk7IR9mPcHG9NmLmWspDxyAEEo4kbsryBKb9Q&u_aref=rfk1VxpAzqpH%2FZvj8AaqAX6zdFA%3D; ssxmod_itna=euG=D57KDKYvxWTxBPGI2nWDRWGoGk3zGDvbbRDBd84iNDnD8x7YDvImLtFI9F9DO2xifeI8ip3dWc7xTYq73f+fGKEGxiTD4q07Db4GkDAqiOD7kRhoD4RKGwD0eG+DD4DWKsK6Z2zx0PD72XRbqNDm4GWlqDgDY=DIqNDAbdKxD3nDAUY6qUKg4DBGgrXleGvecQ2IutSeqGuzWQuYWaF164D5iraGqGyDtFnQL0DukXntKiPNhoKIAv=7q4Fb7Gq87Ee3A4sS/4vbiAQQBxNKGSQlhZP0Qh=mtqxD; ssxmod_itna2=euG=D57KDKYvxWTxBPGI2nWDRWGoGk3zGDvbbD8dpYxGXhqYGaK9f5zHIydGkf1qmLrufC79IxUSWwKeIPfhjxT8gI=Cua8ayGQbRP0A8nPAeuip17SxuGStAdGUVS4idhxaLipeaDctMWrzoYPzkCqWBpDiozuxKF2QSxmmSpKzD=07o+7QOpv+atDpqpBtS1TvgPFIuBT+YE5s6UOploLEa4Wa0+5bARLzMldxa3CzN3PI1FOuED07wx08DY944D=='
cookie = cookie.encode("utf-8").decode("latin1")
cookies ={'Cookie': cookie}

In [35]:
def save_user_img(user_url, user_name, file_path='/Users/shangzizhi/CodeManager/MyPython/Django/helloworld/media/user_img/'):
    response = requests.get(url = user_url,cookies=cookies,headers=headers)
    # print(response.text)
    soup = bs(response.text, features = "lxml")
    # r'<i class="iconfont ic-list-comments"></i>\s*([\d]*)\s*'
    res = r'src="(https://upload.jianshu.io/users/([^"]*))'
    res =re.compile(res)
    # print(res)
    # print(str(soup))
    img_url = re.findall(res,str(soup))
    # print(img_url[0][0])
    save_img(img_url[0][0],file_path+user_name+'.png')
    return "/media/user_img/"+user_name+'.png'
    
def save_img(img_url, file_path=''):
    img = requests.get(img_url,headers=headers,cookies=cookies)
    byte = img.content
    f = open(file_path, 'wb')
    f.write(byte)
    print(file_path, '文件保存成功！')
    f.close()

In [36]:
pre_dataset = os.listdir('./result')
print(len(pre_dataset))
print(pre_dataset[0])

5507
1069


In [38]:
url_list = []
for i in range(0,len(pre_dataset)):
    current_data_dir = './result/'+pre_dataset[i]
    current_html_dir = current_data_dir+'/body.html'
    result_json_dir = current_data_dir + '/result.json'
    target_json_dir = current_data_dir + '/result.json'
    if os.path.exists(result_json_dir) :
        with open(result_json_dir, 'r') as f:
            result_json = json.load(f)
            # print(result)
            result = result_json.get('results')
            url = result['passage_url']
            if url in url_list :
                # print("重复")
                continue
            else:
                url_list += [result['passage_url']]
                if result['body'] == 'None':
                    print(result_json)
                    print(None)
                    pass
                else:
                    # result_json['results']['user_homepage'] = save_user_img(result_json['results']['user_homepage'],result_json['results']['user_name'])
                    result_json = json.dumps(result_json, indent=3, ensure_ascii=False)
                    print(pre_dataset[i])
                    with open(target_json_dir, 'w', encoding='utf-8') as f:
                        f.write(result_json)
                    break
    

1855


In [48]:
import sqlite3
conn = sqlite3.connect("db.sqlite3")
cursor = conn.cursor()
def write_to_sqlite(info):
    sql = ''' insert into article_articlepost
            (title, body, comments, create_time, favors, likes, passage_url, pre_id, tag, abs, author_id, views, author_url, author_img_src, author_followers, author_articles, author_likes)
        values
              (:title, :body, :comments, :create_time, :favors, :likes, :passage_url, :pre_id, :tag, :abs, :author_id, :views, :author_url, :author_img_src, :author_followers, :author_articles, :author_likes)'''
    cursor.execute(sql,{'title':info['passage_title'], 'body':info['body'], 'comments':info['comments'], 'create_time':info['create_time'], 'favors':0, 'likes':info['likes'], 'passage_url':info['passage_url'], 'pre_id':info['user_name'], 'tag': info['tag'], 'abs':info['passage_abs'], 'author_id':1, 'views':info['views'],'author_url':info["user_homepage"], 'author_img_src':info['user_img'], 'author_followers':info['user_info']['粉丝'], 'author_articles':info['user_info']['文章'], 'author_likes': info['user_info']['关注']})
    conn.commit()

In [49]:
pre_dataset = os.listdir('./result')
print(len(pre_dataset))
import json
print(pre_dataset[1])
current_data_dir = './result/'+pre_dataset[1]
current_html = current_data_dir+'/body.html'
result_json = current_data_dir + '/result.json'
if os.path.exists(result_json) :
    with open(result_json, 'r') as f:
        result = json.load(f)
        # print(result)
        result = result.get('results')
print(result)
write_to_sqlite(result)

5507
1855
{'create_time': '2020/08/08 19:19:28', 'views': 145, 'body': '<article class="_2rhmJa"><blockquote>\n<p><b>&lt;!DOCTYPE html&gt;<br/></b></p>\n<p><b>&lt;html&gt;</b></p>\n<p><b>\t&lt;head&gt;</b></p>\n<p><b>\t\t&lt;meta charset="utf-8"&gt;</b></p>\n<p><b>\t\t&lt;title&gt;dom相关节点操作&lt;/title&gt;</b></p>\n<p><br/></p>\n<p><b>\t\t&lt;script src=\'js/jquery/jquery.min.js\'&gt;&lt;/script&gt;</b></p>\n<p><b>\t\t&lt;script type=\'text/javascript\'&gt;</b></p>\n<p><br/></p>\n<p><b>\t\t\t/* DOM操作管理 属性方法 */</b></p>\n<p><b>\t\t\t$(function(){</b></p>\n<p><b>\t\t\t\t/* 在标签内进行添加 追加后面append\xa0 追加前面prepend\xa0 wrap包裹*/</b></p>\n<p><b>\t\t\t\t$(\'h1\').append(\'&lt;i&gt;列表&lt;/i&gt;\')</b></p>\n<p><b>\t\t\t\t$(\'h1\').prepend(\'&lt;i&gt;展示&lt;/i&gt;\')</b></p>\n<p><b>\t\t\t\t$(\'h1\').wrap(\'&lt;h2&gt;包裹&lt;/h2&gt;\')</b></p>\n<p><br/></p>\n<p><b>\t\t\t\t/* 外部 插入到前面before 插入到后面after\xa0 */</b></p>\n<p><b>\t\t\t\t$(\'h1\').before(\'&lt;i&gt;AAA&lt;/i&gt;\')</b></p>\n<p><b>\t\t\t\t$(\'h1\').

In [50]:
url_list = []
for i in range(1,len(pre_dataset)):
    current_data_dir = './result/'+pre_dataset[i]
    current_html_dir = current_data_dir+'/body.html'
    result_json_dir = current_data_dir + '/result.json'
    target_json_dir = current_data_dir + '/result.json'
    if os.path.exists(result_json_dir) :
        with open(result_json_dir, 'r') as f:
            result_json = json.load(f)
            # print(result)
            result = result_json.get('results')
            url = result['passage_url']
            if url in url_list :
                # print("重复")
                continue
            else:
                url_list += [result['passage_url']]
                if result['body'] == 'None':
                    print(result_json)
                    print(None)
                    pass
                else:
                    write_to_sqlite(result)

{'version': '1.0', 'results': {'create_time': '1970/01/01 08:00:00', 'views': 0, 'body': 'None', 'passage_url': 'https://www.jianshu.com/p/1a45a8bd58a6', 'passage_title': '新型冠状病毒世界疫情动态变化地图', 'passage_abs': '动态展示新型冠状病毒在世界范围内的变化趋势', 'user_name': 'Funzion', 'user_homepage': 'https://www.jianshu.com/u/9bd0ab91b1ac', 'comments': 0, 'likes': 3, 'user_info': {'name': 'Funzion', '关注': 17, '粉丝': 61, '文章': 10, '字数': 17, '收获喜欢': 17, '总资产': 17}, 'tag': 'else'}, 'explain': {'used': True, 'details': 'this is for josn test'}}
None
{'version': '1.0', 'results': {'create_time': '1970/01/01 08:00:00', 'views': 0, 'body': 'None', 'passage_url': 'https://www.jianshu.com/p/c1dddc4a4fec', 'passage_title': '秀到起飞！LeetCode官方推出算法面试指导手册（代码版）限时开源', 'passage_abs': '什么是LeetCode？ leetcode是个题库，里面有很多编程多面试的题目，可以在线编译运行。难度比较高。如果自己能做出来，对面大公司很有帮...', 'user_name': '愿天堂没有BUG', 'user_homepage': 'https://www.jianshu.com/u/628bedc62e07', 'comments': 0, 'likes': 0, 'user_info': {'name': '愿天堂没有BUG', '关注': 24, '粉丝': 5, '文章': 166, 